## Importing libraries

In [1]:
import numpy as np
from dlordinal.losses import TriangularCrossEntropyLoss
from dlordinal.datasets import FGNet
from torch import cuda, nn
from torch.optim import Adam
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor
from skorch import NeuralNetClassifier


## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the `DataLoader`, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [19]:
optimiser_params = {
    'lr': 1e-3,
    'bs': 400,
    'epochs': 5,
    's': 2,
    'c': 0.2,
    'beta': 0.5
}

workers = 3

Now we use the `FGNet` method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the `StratifiedShuffleSplit` method. Finally, with all the partitions, we load the images using a method called `DataLoader`.

In [20]:
fgnet = FGNet(root="./datasets/fgnet", download=True, process_data=True)

train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

num_classes = len(train_data.classes)
classes = train_data.classes
targets = train_data.targets

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cuda device


## Estimator

We are setting up a deep learning model using `PyTorch` and `Skorch`. First, we define the model architecture using ResNet18, a pre-trained convolutional neural network, and customize its fully connected layer to match the number of classes in our classification task. Then we specify the loss function, in this case, a custom Triangular Cross Entropy Loss[1]. Finally, we configure the Skorch estimator, which serves as a bridge between PyTorch and scikit-learn, allowing us to train and evaluate our model seamlessly. We provide the model, loss function, and optimiser details such as the learning rate and number of epochs to the estimator. Additionally, we specify parameters for data loading and processing, like batch size and the number of workers, to optimise training performance.

[1]: Víctor Manuel Vargas, Pedro Antonio Gutiérrez, Javier Barbero-Gómez, and César Hervás-Martínez (2023). *Soft Labelling Based on Triangular Distributions for Ordinal Classification.* Information Fusion, 93, 258--267. doi.org/10.1016/j.inffus.2023.01.003

In [21]:
# Model
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss function
loss_fn = TriangularCrossEntropyLoss(num_classes=num_classes).to(device)

# Skorch estimator
estimator = NeuralNetClassifier(
    module=model,
    criterion=loss_fn,
    optimizer=Adam,
    lr=optimiser_params["lr"],
    max_epochs=optimiser_params["epochs"],
    train_split=None,
    callbacks=[],
    device=device,
    verbose=0,
    iterator_train__batch_size=optimiser_params["bs"],
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers - 1,
    iterator_train__pin_memory=True,
    iterator_valid__batch_size=optimiser_params["bs"],
    iterator_valid__shuffle=False,
    iterator_valid__num_workers=workers - 1,
    iterator_valid__pin_memory=True,
)

In [22]:
# estimator.fit(X=train_data, y=stargets)
targets = np.array(targets)
estimator.fit(X=train_data, y=targets)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(6

In [23]:
train_probs = estimator.predict_proba(train_data)
print(f"Train probabilities = {train_probs=}\n")

test_probs = estimator.predict_proba(test_data)
print(f"Test probabilities = {test_probs=}")

Train probabilities = train_probs=array([[ 3.7356095 ,  2.280471  ,  0.27906656, -6.5274134 , -0.1423619 ,
        -0.89688575],
       [ 6.7312865 ,  4.2798862 , -2.754112  , -7.015324  , -0.58337   ,
        -1.614481  ],
       [ 1.5178611 ,  0.3035042 , -1.0939833 , -1.1187612 ,  0.3635073 ,
        -1.4568175 ],
       ...,
       [-8.981531  , -2.1939955 , -1.2311378 , -1.599317  ,  1.7429321 ,
         8.956122  ],
       [-7.570979  , -2.4199474 , -0.9986418 ,  2.073321  ,  1.8904057 ,
         3.514359  ],
       [-4.612633  , -2.3110492 ,  1.4501587 ,  1.0073776 ,  0.30610457,
         1.1957583 ]], dtype=float32)

Test probabilities = test_probs=array([[-0.7816221 , -0.87308043, -1.196569  , -2.6637518 ,  1.0128176 ,
         2.083984  ],
       [-0.04164401,  1.2640952 ,  1.5022627 , -2.0729616 , -0.1945019 ,
        -1.6816527 ],
       [ 7.281721  ,  2.9113057 , -3.4834485 , -7.3575487 ,  1.2093832 ,
        -1.4325407 ],
       ...,
       [-9.944385  , -3.6944542 , -0.4